In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy import stats
import sympy

### Obtaining the data: 
Archives "Pk-fiducial-z_0_.dat" and "Pknw-fiducial-z_0_.dat" \
Columns: $k$ , $Pk/ s_8^2$ ,  $s_8$

In [22]:
# f and D for each z_mean:
f = [0.8790616234808487, 0.9052940279841754, 0.9248305916607235, 0.942646864519483]
D = [0.6051299241324081, 0.5557769195555776, 0.5132266632166314, 0.46785418122063316]

# Centered position of each bin
z_i = [1., 1.2, 1.4, 1.65]

Pk_00 = np.loadtxt("Pk-fiducial-z_00.dat")
Pk_01 = np.loadtxt("Pk-fiducial-z_01.dat")
Pk_02 = np.loadtxt("Pk-fiducial-z_02.dat")
Pk_03 = np.loadtxt("Pk-fiducial-z_03.dat")

''' 
Sólo s8 está al cuadrado cierto? no todo?
'''
k_00 = Pk_00[:,0]
s8_00 = Pk_00[:,2]
P_dd00 = Pk_00[:,1] * (Pk_00[:,2]**2)

k_01 = Pk_01[:,0]
s8_01 = Pk_01[:,2]
P_dd01 = Pk_01[:,1] * (Pk_01[:,2]**2)

k_02 = Pk_02[:,0]
s8_02 = Pk_02[:,2]
P_dd02 = Pk_02[:,1] * (Pk_02[:,2]**2)

k_03 = Pk_03[:,0]
s8_03 = Pk_03[:,2]
P_dd03 = Pk_03[:,1] * (Pk_03[:,2]**2)

def P_dd(k, u, zi):
    if zi == zi[0]:
        index_k = k_00.index(k)
        return P_dd00[index_k]
    elif zi == zi[1]:
        index_k = k_01.index(k)
        return P_dd01[index_k]
    elif zi == zi[2]:
        index_k = k_02.index(k)
        return P_dd02[index_k]
    else:
        index_k = k_03.index(k)
        return P_dd03[index_k]



# For Pk_nw (no wiggled power spectrum)
Pk_nw_00 = np.loadtxt("Pk-nw-fiducial-z_00.dat")
Pk_nw_01 = np.loadtxt("Pk-nw-fiducial-z_01.dat")
Pk_nw_02 = np.loadtxt("Pk-nw-fiducial-z_02.dat")
Pk_nw_03 = np.loadtxt("Pk-nw-fiducial-z_03.dat")

''' 
Sólo s8 está al cuadrado cierto? no todo?
'''
k_nw_00 = Pk_nw_00[:,0]
s8_nw_00 = Pk_nw_00[:,2]
P_dd_nw00 = Pk_nw_00[:,1] * (Pk_nw_00[:,2]**2)

k_nw_01 = Pk_nw_01[:,0]
s8_nw_01 = Pk_nw_01[:,2]
P_dd_nw01 = Pk_nw_01[:,1] * (Pk_nw_01[:,2]**2)

k_nw_02 = Pk_nw_02[:,0]
s8_nw_02 = Pk_nw_02[:,2]
P_dd_nw02 = Pk_nw_02[:,1] * (Pk_nw_02[:,2]**2)

k_nw_03 = Pk_nw_03[:,0]
s8_nw_03 = Pk_nw_03[:,2]
P_dd_nw03 = Pk_nw_03[:,1] * (Pk_nw_03[:,2]**2)


# Based on table 3 of the page 14:
n = [6.86 * 10**(-4), 5.58 * 10**(-4), 4.21* 10**(-4), 2.61 * 10**(-4)]
V_s = [7.94, 9.15, 10.05, 16.22]
#galazy bias factor
b = [1.46, 1.61, 1.75, 1.9]

Parámetros a encontrar:
$D_A(z_i)$, $H(z_i)$, $f\sigma_8(z_i)$, $b\sigma_8$ , $P_s(z_i)$, $\omega_{b,0}$, $h$, $\omega_{m,0}$, $n_s$, $\sigma_v (z_{mean})$ y $\sigma_p (z_{mean})$


In [ ]:
#Fiducial parameters
Om_b0 = 0.022445
h = 0.67
H0 = 100 * h

Om_m0 = 0.143648
n_s = 0.96
#Qué es As10^9?
A_s9 = 2.12605
# Reference amplitud As
# Donde se usa? y por qué se especifica el A_s^9
A_s = 2.1260500000000005e-9
s8 = 0.815583
''' 
Cuáles son los non-pyhsical base and physical?
'''

# Corresponding non-physical base parameters: {Omegab,  Omegac, h, 10^9 As} :> {0.05000000000000001, 0.2671256676696824, 0.67, 2.1260500000000007}
# Corresponding physical base parameters: {omegab,  omegac, h, 10^9 As} :> {0.022445000000000007, 0.11991271221692044, 0.67, 2.1260500000000007}
# Halofit version in CAMB: 4
# Maximum k value in CAMB (h/Mpc): 5
# Numeric CAMB parameters : {AccuracyBoost -> 2, TransferHighPrecision -> True, TransferKperLogInt -> 50}
# Constant Neutrino CAMB parameters : {OmegaNu -> 0.00143717, MasslessNeutrinos -> 2.046, MassiveNeutrinos -> 1, NuMassFractions -> {1}}
Om_nu = 0.00143717
# List of z points: {1., 1.2, 1.4, 1.65}

''' 
Qué son los parameters variations?
'''


In [5]:
''' 
Como aplico éstas restricciones??
'''

# Aplicando la restricción E(0) = 1
#np.sqrt((Om_m0) + Om_DE0  +  (Om_k0)) = 1
Om_DE0 = 0.68 
Om_k0= 1 - Om_m0 - Om_DE0

# Se definen los parámetros de referencia

# Reference parameters to evaluate P_obs on them
#bs8, fs8, f, sp, s8, H, D_A

c = 299792458
bs8 = b * s8
fs8 = f * s8

w_0 = -1
w_a = 0

A_surv = 15000
s_z = 0.0001 #Spectroscopic redshift error

''' 
Dice que Ps es 0 at reference cosmology pero cuál es su expresión?
'''
P_s = 0

In [6]:
# cómo defino H_ref y D_A_ref y cuál es la diferencia de H(z)?
def E(z):
    return np.sqrt((Om_m0*(1+z)**(3)) + Om_DE0 +  (Om_k0*(1+z)**2))

def H(z):
    return H0 * E(z)

def r(z):
    a = 1/E(z)
    return (c/H0) * np.cumsum(a, dtype=float)  

def D_A(z):
    if Om_k0 < 0:
        return (1+z)**(-1) * (c/H0) * (1/ np.sqrt(-Om_k0))* np.sin(np.sqrt(-Om_k0) * (H0/c) * r(z))
    elif Om_k0 == 0:
        return (1+z)**(-1) * r(z)
    else:
        return (1+z)**(-1) * (c/H0) * (1/ np.sqrt(Om_k0))* np.sinh(np.sqrt(Om_k0) * (H0/c) * r(z))


In [7]:

''' 
Esto no es necesario cierto?
'''
#gamma = 0.55
#def Om_m(z):
#    return Om_m0 * ((1+z)**3) / (E(z)**2)

#def f(z):
#  return Om_m(z)**gamma

def s_r(z):
  return (c/H(z)) * s_z

def F_z(k,u,z):
  return np.exp(-k**2 * u**2 * s_r(z)**2)

def qpar(z, H = None):
  H_value = H(z) if H is None else H
  return H_ref(z)/H_value

def qper(z, D_A = None):
  D_A_value = D_A(z) if D_A is None else D_A
  return D_A_value/ D_A_ref(z)

''' 
Cuáles son los k de la tabla? son los k de referencia? Cuáles son los mu y k de ref?
En ese caso, cómo saco mu?
K y mu dependen de z? Es que q depende de z pero no está así en la ec.
'''

# To convert the known reference k_ref and mu_ref to the unknown, true k and mu
def k(z, k_ref, u_ref, H = None, D_A = None):
  qper = qper(z, D_A = None)
  qpar = qpar(z, H = None)
  return (k_ref /qper)* (1 + u_ref**2 * ((qper**2/qpar**2) -1))**(1/2)

def u(z, u_ref):
  qper = qper(z, D_A = None)
  qpar = qpar(z, H = None)
  return u_ref * (qper/qpar) * (1 + u_ref**2 * ((qper**2/qpar**2) -1))**(-1/2)

# Non-linear damping factor of the BAO signal.
def g_u(u, z):
  return sv(z)**2 *(1- u**2 + (u**2)*(1+ f(z))**2)

def P_dw(k, u, z):
  return P_dd(k, u, z)* np.exp(-g_u(u,z)*k**2) + P_nw(k,u,z)* (1- np.exp(-g_u(u,z)*k**2))

# In practice:
# sv(z) controls the strength of the non-linear damping of the BAO signal in all directions.
def sv(z):
  return sv(z_mean) * D(z) / D(z_mean)

def sp(z):
  return sp(z_mean) * D(z) / D(z_mean)




In [8]:
# Full non-linear model
def P_obs(z, bs8=None, fs8=None, f= None , sp=None, s8= None, H= None, D_A = None):
  bs8_value = bs8(z) if bs8 is None else bs8
  fs8_value = fs8(z) if fs8 is None else fs8
  f_value = f(z) if f is None else f
  sp_value = sp(z) if sp is None else sp
  s8_value = s8(z) if s8 is None else s8
  
  H_value = None if H is None else H
  D_A_value = None if D_A is None else D_A
  
  k_eval = k(k_ref, u_ref, H = H_value, D_A = D_A_value)
  u_eval = u(k_ref, u_ref, H = H_value, D_A = D_A_value)

  return (1/(qpar(z)**2 * qper(z)))*((bs8_value + fs8_value*u_eval**2)**2/(1+(f_value*k_eval*u_eval*sp_value)**2)) * (P_dw(k_eval,u_eval,z)/s8_value**2)* F_z(k_eval,u_eval,z) + P_s(z)


def V_eff(zi, k, u):
  bin = z_i.index(zi)
  return V_s_zmean[bin] * (n_zmean[bin])*P_obs(k,u, zi)/(n_zmean*P_obs(k,u, zi) + 1)**2

In [ ]:
# epsilon varía entre los 0.0001 y 0.01
def derivada_ln_P_obs(k, u, z, a, eps =1e-4):
    #h: Tamaño del paso (opcional, por defecto 1e-5).
    '''
    Cómo  hago si s8 depende de k?
    cuales son los límites de la integral para sv?
    '''
    if a == 'P_s':
      ln_P_obs = np.log(P_obs)
      derivada = 1/ ln_P_obs(k, u, z)

    # if are shape parameters w_b0 , h, w_m0 , n_s
    elif a == 'w_b0' or a== 'h' or a == 'w_m0' or a == 'n_s':
      # str to function
      a= globals().get(a)
      derivada = ( np.log(P_dw(k,z, a = a*(1+ eps))/s8(z)**2)- np.log(P_dw(k,z, a = a *(1-eps))/s8(z)**2) )/ (2 * eps* a)

    # if are non-linear parameters sv y sp
    #three-point stencil derivative of f with respect to x at x0
    elif a == 'sv':
      a= globals().get(a)
      derivada = (np.log(P_dw(k, u, z, sv = sv*(1+ eps))/ s8(z)**2)- np.log(P_dw(k, u, z, sv = sv*(1 - eps))/ s8(z)**2)) / (2 * eps * np.log(P_dw(k, u, z, sv = sv)/ s8(z)**2))

    elif a == 'sp':
      a= globals().get(a)
      derivada = (np.log(1/ (1+(f(z)*k*u*sp(z)*(1+ eps))**2))- np.log(1/ (1+(f(z)*k*u*sp(z)*(1- eps))**2))) / (2 * eps * np.log(1/ (1+(f(z)*k*u*sp(z))**2)))

    # if parameters are fs8 and bs8
    elif a == 'fs8':
      a= globals().get(a)
      derivada = (np.log(P_obs(k,u,z, fs8 = fs8**(1 + eps))) - np.log(P_obs(k,u,z, fs8 = fs8**(1 - eps)))) / (2* eps * np.log(fs8))

    elif a == 'bs8':
      a= globals().get(a)
      derivada = (np.log(P_obs(k,u,z, bs8 = bs8**(1 + eps))) - np.log(P_obs(k,u, z, bs8 = bs8**(1 - eps)))) / (2* eps * np.log(bs8))


    elif a == 'H':
      a= globals().get(a)
      derivada = 8* (np.log(P_obs(k,u,z, H= H**(1+ eps)))- np.log(P_obs(k, u, z,H= H**(1- eps))))/(12* eps * np.log(H)) - (np.log(P_obs(k,u,z, H= H**(1+ 2*eps)))- np.log(P_obs(k, u, z, H= H**(1 - 2*eps))))/(12* eps * np.log(H))
    elif a == 'D_A':
      a= globals().get(a)
      derivada = 8* (np.log(P_obs(k,u,z, D_A = D_A**(1+ eps)))- np.log(P_obs(k, u, z, D_A = D_A**(1- eps))))/(12* eps * np.log(D_A)) - (np.log(P_obs(k,u,z, D_A = D_A**(1+ 2*eps)))- np.log(P_obs(k, u, z, D_A = D_A**(1 - 2*eps))))/(12* eps * np.log(D_A))

    return derivada



In [5]:
#ejemplo de la derivada anterior
def y(z):
  return z**2

def f(z):
  y_value = y(z)
  return y_value

def derivada_f(z, h=1e-5):
    #h: Tamaño del paso (opcional, por defecto 1e-5).

    derivada = (f(z + h*z) - f(z - h*z)) / (2 * h * z)
    return derivada

derivada = derivada_f(3)
print(derivada)

6.000000000039306


In [6]:
'''
Se debe integrar numéricamente con trapecios o simpson?
Lo hice con trapecios, está bien?
'''
#romber
#cumsum

k_min = 0
k_max = 1
#ver parámetros reales

a = 'bs8'
b = 'f'

#definimos los parámetros con los cuales derivar
k = np.linspace(k_min, k_max, 100)
u = np.linspace(-1, 1, 100)
def ec1(k, u, zi, a, b):
  return k**2 * derivada_ln_P_obs(k, u, zi, a) *derivada_ln_P_obs(k, u, zi, b)* V_eff(zi, k, u)

# Realizar la integración utilizando la regla del trapecio
ec2 = np.trapz(ec1, k)

# Segunda integral
ec3 = np.trapz(ec2, u)

print("El resultado de la integral es:", ec3)

NameError: name 'np' is not defined